In [35]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests        
import json           
import pandas as pd    
import numpy as np    
import matplotlib.pyplot as plt 
import datetime as dt  

In [36]:
from backtesting.test import SMA, GOOG
# GOOG.to_csv("data/GOOG.csv")

In [65]:
def get_bars(symbol, interval = '1h'):
   root_url = 'https://api.binance.com/api/v1/klines'
   filename = "data/"+symbol+"_"+interval+".csv"
   url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
   print(url)
   columns = ['open_time','Open', 'High', 'Low', 'Close', 'Volume',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
   if os.path.exists(filename):
        data = pd.read_csv(filename, sep=';')
        df = pd.DataFrame(data,columns=columns)
        print(dt.datetime.fromtimestamp(df['open_time'].max()/1000.0))
   else:   
        data = json.loads(requests.get(url).text)        
        df = pd.DataFrame(data,columns=columns)
        df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
        df.sort_values(["open_time"],ascending=False)
        df = df.astype('double')
        df['Volume'] = df['Volume'].astype('int')
        df = df.drop(['qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1)
        df.to_csv(filename, sep=';')

   return df

In [66]:
dataframe = get_bars("BTCUSDT", "1h")

https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1h&limit=1001
2021-10-23 12:00:00


In [51]:
class SmaCross(Strategy):
    n1=0
    n2=0

        
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            
    def setInitialParams(self,valor1, valor2):
        self.n1 = valor1;
        self.n2 = valor2;

In [52]:
maiorRetorno = {}
maoirAcerto = {}
tamanho = 30
for s1 in range(tamanho):
    for s2 in range(tamanho):
        print("testando nos valores ", s1, " e ", s2)
        SmaCross.setInitialParams(SmaCross,s1, s2)
        bt = Backtest(dataframe, SmaCross,cash=100000000,exclusive_orders=True)
        output = bt.run()
        if maiorRetorno=={}:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if maoirAcerto=={}:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
        if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}        

testando nos valores  0  e  0
testando nos valores  0  e  1
testando nos valores  0  e  2
testando nos valores  0  e  3
testando nos valores  0  e  4
testando nos valores  0  e  5
testando nos valores  0  e  6
testando nos valores  0  e  7
testando nos valores  0  e  8
testando nos valores  0  e  9
testando nos valores  0  e  10
testando nos valores  0  e  11
testando nos valores  0  e  12
testando nos valores  0  e  13
testando nos valores  0  e  14
testando nos valores  0  e  15
testando nos valores  0  e  16
testando nos valores  0  e  17
testando nos valores  0  e  18
testando nos valores  0  e  19
testando nos valores  0  e  20
testando nos valores  0  e  21
testando nos valores  0  e  22
testando nos valores  0  e  23
testando nos valores  0  e  24
testando nos valores  0  e  25
testando nos valores  0  e  26
testando nos valores  0  e  27
testando nos valores  0  e  28
testando nos valores  0  e  29
testando nos valores  1  e  0
testando nos valores  1  e  1
testando nos valores

testando nos valores  8  e  29
testando nos valores  9  e  0
testando nos valores  9  e  1
testando nos valores  9  e  2
testando nos valores  9  e  3
testando nos valores  9  e  4
testando nos valores  9  e  5
testando nos valores  9  e  6
testando nos valores  9  e  7
testando nos valores  9  e  8
testando nos valores  9  e  9
testando nos valores  9  e  10
testando nos valores  9  e  11
testando nos valores  9  e  12
testando nos valores  9  e  13
testando nos valores  9  e  14
testando nos valores  9  e  15
testando nos valores  9  e  16
testando nos valores  9  e  17
testando nos valores  9  e  18
testando nos valores  9  e  19
testando nos valores  9  e  20
testando nos valores  9  e  21
testando nos valores  9  e  22
testando nos valores  9  e  23
testando nos valores  9  e  24
testando nos valores  9  e  25
testando nos valores  9  e  26
testando nos valores  9  e  27
testando nos valores  9  e  28
testando nos valores  9  e  29
testando nos valores  10  e  0
testando nos valor

testando nos valores  17  e  21
testando nos valores  17  e  22
testando nos valores  17  e  23
testando nos valores  17  e  24
testando nos valores  17  e  25
testando nos valores  17  e  26
testando nos valores  17  e  27
testando nos valores  17  e  28
testando nos valores  17  e  29
testando nos valores  18  e  0
testando nos valores  18  e  1
testando nos valores  18  e  2
testando nos valores  18  e  3
testando nos valores  18  e  4
testando nos valores  18  e  5
testando nos valores  18  e  6
testando nos valores  18  e  7
testando nos valores  18  e  8
testando nos valores  18  e  9
testando nos valores  18  e  10
testando nos valores  18  e  11
testando nos valores  18  e  12
testando nos valores  18  e  13
testando nos valores  18  e  14
testando nos valores  18  e  15
testando nos valores  18  e  16
testando nos valores  18  e  17
testando nos valores  18  e  18
testando nos valores  18  e  19
testando nos valores  18  e  20
testando nos valores  18  e  21
testando nos valor

testando nos valores  26  e  13
testando nos valores  26  e  14
testando nos valores  26  e  15
testando nos valores  26  e  16
testando nos valores  26  e  17
testando nos valores  26  e  18
testando nos valores  26  e  19
testando nos valores  26  e  20
testando nos valores  26  e  21
testando nos valores  26  e  22
testando nos valores  26  e  23
testando nos valores  26  e  24
testando nos valores  26  e  25
testando nos valores  26  e  26
testando nos valores  26  e  27
testando nos valores  26  e  28
testando nos valores  26  e  29
testando nos valores  27  e  0
testando nos valores  27  e  1
testando nos valores  27  e  2
testando nos valores  27  e  3
testando nos valores  27  e  4
testando nos valores  27  e  5
testando nos valores  27  e  6
testando nos valores  27  e  7
testando nos valores  27  e  8
testando nos valores  27  e  9
testando nos valores  27  e  10
testando nos valores  27  e  11
testando nos valores  27  e  12
testando nos valores  27  e  13
testando nos valor

In [53]:
print(maiorRetorno)
# print(maoirAcerto)

{'sma1': 9, 'sma2': 1, 'output': Start                     2021-09-11 19:59...
End                       2021-10-23 12:59...
Duration                     41 days 17:00:00
Exposure Time [%]                          99
Equity Final [$]                  1.61902e+08
Equity Peak [$]                    1.6276e+08
Return [%]                            61.9018
Buy & Hold Return [%]                 36.0511
Return (Ann.) [%]                     5873.18
Volatility (Ann.) [%]                 4275.49
Sharpe Ratio                          1.37369
Sortino Ratio                         159.602
Calmar Ratio                          407.706
Max. Drawdown [%]                    -14.4054
Avg. Drawdown [%]                    -1.89205
Max. Drawdown Duration       12 days 23:00:00
Avg. Drawdown Duration        0 days 23:00:00
# Trades                                  217
Win Rate [%]                          76.9585
Best Trade [%]                        5.50588
Worst Trade [%]                      -9.04762
A

In [54]:
# SmaCross.setInitialParams(SmaCross,12, 15)
# bt = Backtest(GOOG, SmaCross,cash=1000,exclusive_orders=True)
# output = bt.run()
# print(output)
SmaCross.setInitialParams(SmaCross,2, 6)
bt = Backtest(dataframe, SmaCross,cash=1000000)
output = bt.run()
output
# bt.plot()

Start                     2021-09-11 19:59...
End                       2021-10-23 12:59...
Duration                     41 days 17:00:00
Exposure Time [%]                          99
Equity Final [$]                       706705
Equity Peak [$]                    1.0207e+06
Return [%]                           -29.3295
Buy & Hold Return [%]                 36.0511
Return (Ann.) [%]                    -94.7486
Volatility (Ann.) [%]                 4.20104
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                     -33.072
Avg. Drawdown [%]                    -14.6162
Max. Drawdown Duration       31 days 17:00:00
Avg. Drawdown Duration       13 days 19:00:00
# Trades                                  206
Win Rate [%]                          27.1845
Best Trade [%]                        9.04762
Worst Trade [%]                      -5.50588
Avg. Trade [%]                    